# OpenQA Job Template Helper

This notebook helps with doing bulk operations on OpenQA job templates such as copying or moving the contents of a job group.

In [ ]:
include("../src/init.jl")

using JDP.Trackers.OpenQA

## Product defs map

Create a product definition map for the different architectures. Make sure to set the desired product version.

In [ ]:
prods = OpenQA.get_json(OpenQA.osd, "products")["Products"]
prods[1]

In [ ]:
aprods = map(filter(prods) do p
        p["flavor"] == "Installer-DVD" && p["distri"] == "sle" && p["version"] == "15-SP1"
    end) do p
        p["arch"] => p["id"]
    end |> Dict

In [ ]:
jobids = OpenQA.get_json(OpenQA.osd, "jobs?group_id=155")

## Job template definitions

Get the current definitions, filter them and map them to a tuple of new templates (but don't actually sumbit them yet to OpenQA).

In [ ]:
jobs = OpenQA.get_json(OpenQA.osd, "job_templates?group_id=155")["JobTemplates"]
jobs[1]

In [ ]:
jobs = OpenQA.get_json(OpenQA.osd, "job_templates")["JobTemplates"]
jobs[1]

In [ ]:
spvmjobs = filter(jobs) do j
    j["machine"]["name"] == "ppc64le-pvm"
end

In [ ]:
foreach(display, spvmjobs)

In [ ]:
spvmjobs[1]["product"]

In [ ]:
nujobs = map(filter(jobs) do j
    startswith(j["test_suite"]["name"], "xfstests")      
end) do j
    (id = j["id"],
     product_id = aprods[j["product"]["arch"]],
     machine_id = j["machine"]["id"],
     test_suite_id = j["test_suite"]["id"],
     name = j["test_suite"]["name"])
end

## Sumbitting changes

Send the changes to OpenQA

In [ ]:
client_script = `openqa-client --host openqa.suse.de --apikey C6C16EEAC002010A --apisecret A411FEDE926A5A9A`

Delete the templates we queried earlier

In [ ]:
fails = []

for j in nujobs
    try
        run(`$client_script job_templates/$(j.id) delete`)
    catch e
        push!(fails, (error = e, job = j))
    end
end

Create new templates from what we queried earlier

In [ ]:
fails = []

for j in nujobs
    try
        run(`$client_script job_templates post product_id=$(j.product_id) machine_id=$(j.machine_id) test_suite_id=$(j.test_suite_id) group_id=116`)
    catch e
        push!(fails, (error = e, job = j))
    end
end

If there were failures then we can try to find out why. Usually it is because some test suite is already being run for a particular product, but in a different group.

In [ ]:
ftemplates = map(fails) do f
    jid = f.job.test_suite_id
    OpenQA.get_json(OpenQA.osd, "job_templates?test_suite_id=$jid")["JobTemplates"]
end

In [ ]:
for f in ftemplates
    println(f[1]["test_suite"]["name"], " => ")
    for ff in f
        if ff["product"]["version"] == "15-SP1"
            println("\t", ff["group_name"])
        end
    end
end
    